In [126]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import icecream as ic
import sys
import os
import subprocess

sys.path.append('../EGSL/')
import egsl

%load_ext autoreload
%autoreload 2

# paths
inpath = '../00_Datasets/' 
gsl_path = 'C:/Users/fanie/GSL_exe/'
gsl_working = '/gsl/'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [127]:
# Import pegm drillhole data

collars= pd.read_csv(inpath + 'collars_pegm.csv')
survey = pd.read_csv(inpath + 'survey_pegm.csv')
survey['dip'] = abs(survey['dip'])
lith = pd.read_csv(inpath + 'lith_pegm.csv')
assays = pd.read_csv(inpath + 'assay_pegm.csv')

# Prepare
# initialise gsl object
gsl = egsl.GSL(gsl_path, gsl_working)

# initialise collar objectS
collar = egsl.Collar(collars,'DHID','x','y','z')
collar.to_gsl(gsl.working_path+'collars.gsl')

# initialise survey object
surv = egsl.Survey(survey, 'DHID', 'dip', 'azimuth', 'depth')  
surv.map_dhid(collar.dhid_map)
surv.to_gsl(gsl.working_path+'survey.gsl')    

# initialise assay object
# Check for duplicates
dup = assays[assays.duplicated(['DHID','from','to'],keep = False)]
dup = dup['DHID'].unique()
assays = assays[~assays['DHID'].isin(dup)] 

assay=egsl.Interval(assays,'DHID','from','to')
# Check for gaps and overlaps - compositing in GSL will not work with gaps or overlaps
assay_filled =assay.fill_gaps()
assay_filled.map_dhid(collar.dhid_map)
assay_filled.to_gsl(gsl.working_path+'assay.gsl')



# Compositing using GSL **composite**

In [128]:
gsl.run('compositing','compositing.par')

Standard Output: 
 -----------------------------------------------------------------------------
                               Compositing                                   

                            Version: 1.3.1
-----------------------------------------------------------------------------

Which parameter file do you want to use?
-----------------------------------------------------------------------------

 INPUT PARAMETERS 

  data file                   = assay.gsl                               
   columns (ID,FROM,TO)       =    4   1   2
   number of variables        =    1
   column for variables        =    3
  var. type (0-cont, 1-cat)   =    0
  trimming limits (min, max)  =  -0.10E+04  0.10E+22
  composite target size and minimum size fraction =    3.00   0.20
  minimum non-null data fraction              =    1.00
  output file                 = assay_comps.gsl                         

-----------------------------------------------------------------------------

 RE

0

# Desurvey using GSL **dir2car**   
* D. S. F. Silva and J. B. Boisvert, 2014, Two New Tools: Directional Survey to GSLIB XYZ Format and Drill Hole Spacing,CCG Paper 2014-404, Centre for Computational Geostatistics, University of Alberta, Edmonton, Canada

In [129]:
survey.head()

,DHID,depth,azimuth,dip,dhid_n
0,DH_000,0.0,90.000,60.000,0
1,DH_000,5.0,89.055,59.902,0
2,DH_000,10.0,89.841,60.300,0
3,DH_000,15.0,90.463,60.785,0
4,DH_000,20.0,91.273,61.225,0


In [130]:
gsl.run('dir2car', 'dir2car.par')

Standard Output: 
 -----------------------------------------------------------------------------
                  Directional to Cartesian Coordinates Conversion            

                            Version:    0.001
-----------------------------------------------------------------------------

Which parameter file do you want to use?
-----------------------------------------------------------------------------

 INPUT PARAMETERS 

  assay data file             = assay_comps.gsl                         
   columns (ID,MD)            =    1   2
   number of variables        =    1
   columns for variables      =    4
   var. type  (0-Con. 1-Cat.) =    0
   trimming limits (min, max) =  -0.10E+01  0.10E+22
  collar data file            = collars.gsl                             
   columns (ID, X, Y and Z)   =    4   1   2   3
  survey data file            = survey.gsl                              
   columns (ID, MD, DIP, AZM) =    4   1   3   2
  output file (xyz coord.)    = compo

0

In [131]:
# Composites to pandas 
comps = egsl.gsl2df(gsl.working_path+'composites.gsl')

In [132]:
comps

,ID,X,Y,Z,MD,CMP-grade
0,0.0,622.1853,1245.9839,13.8792,1.50,0.1000
1,0.0,623.6879,1245.9988,11.2826,4.50,0.1000
2,0.0,625.1884,1246.0191,8.6849,7.50,0.1000
3,0.0,626.6783,1246.0274,6.0810,10.50,0.1000
4,0.0,628.1558,1246.0251,3.4701,13.50,25.5437
...,...,...,...,...,...,...
31174,408.0,694.2053,249.4729,-120.3215,132.03,309.5462
31175,408.0,695.1727,249.3984,-123.1603,135.03,107.5639
31176,408.0,696.1393,249.3193,-125.9992,138.03,34.4941
31177,408.0,697.1064,249.2360,-128.8379,141.03,37.6353


In [133]:
comps.describe()

,ID,X,Y,Z,MD,CMP-grade
count,31179.000000,31179.000000,31179.000000,31179.000000,31179.000000,31179.000000
mean,202.038231,688.501266,815.343252,-128.465633,152.676766,-238.208236
std,118.873925,209.500678,422.384173,97.312515,103.668327,576.436441
min,0.000000,9.785700,-1.822200,-469.361100,1.500000,-999.000000
25%,103.000000,542.268350,449.727500,-190.520600,69.500000,-999.000000
50%,195.000000,697.349100,842.302600,-111.147600,135.500000,12.004900
75%,303.000000,845.570450,1198.146000,-50.239950,220.500000,123.050200
max,408.000000,1327.831700,1500.006700,23.121600,525.540000,4264.239300


In [134]:
import pyvista as pv
from pyvista import examples
pv.set_jupyter_backend('trame')
pv.global_theme.font.size = 10

In [135]:
# plot_array = comps[['X','Y','Z',]].to_numpy()
# dh_plot =pv.plot(plot_array,
#         scalars = comps['CMP-grade'],
#         cmap = 'jet',
#         point_size = 2,
#         log_scale = True
#         )
